In [3]:
import random
import numpy as np

class Network(object):

    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        
        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)
            
        training_data = list(training_data)
        n = len(training_data)
        xrange=range 
        
        for j in xrange(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in xrange(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Epoch {0}: {1} / {2}".format(j, self.evaluate(test_data), n_test))
            else:
                print ("Epoch {0} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):
        xrange=range
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        xrange=range
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        for l in xrange(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        return (output_activations-y)

#### Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))



In [12]:
import pickle
import gzip

def load_data():
    f = gzip.open('./data/mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding="latin1")
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e


In [13]:
training_data , validation_data , test_data = load_data_wrapper()

In [48]:
net = Network([784, 30, 10])
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0: 8242 / 10000
Epoch 1: 8429 / 10000
Epoch 2: 8465 / 10000
Epoch 3: 8456 / 10000
Epoch 4: 8524 / 10000
Epoch 5: 8523 / 10000
Epoch 6: 8498 / 10000
Epoch 7: 8522 / 10000
Epoch 8: 8553 / 10000
Epoch 9: 8572 / 10000
Epoch 10: 8567 / 10000
Epoch 11: 8562 / 10000
Epoch 12: 8571 / 10000
Epoch 13: 8529 / 10000
Epoch 14: 8582 / 10000
Epoch 15: 8602 / 10000
Epoch 16: 8587 / 10000
Epoch 17: 8595 / 10000
Epoch 18: 8603 / 10000
Epoch 19: 8606 / 10000
Epoch 20: 8608 / 10000
Epoch 21: 9460 / 10000
Epoch 22: 9487 / 10000
Epoch 23: 9448 / 10000
Epoch 24: 9443 / 10000
Epoch 25: 9470 / 10000
Epoch 26: 9506 / 10000
Epoch 27: 9496 / 10000
Epoch 28: 9476 / 10000
Epoch 29: 9456 / 10000


In [53]:
training_data , validation_data , test_data = load_data_wrapper()
net = Network([784, 10])            #####remove hidden layer (only input layer and output layer)
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0: 6824 / 10000
Epoch 1: 7586 / 10000
Epoch 2: 9070 / 10000
Epoch 3: 9077 / 10000
Epoch 4: 9130 / 10000
Epoch 5: 9122 / 10000
Epoch 6: 9120 / 10000
Epoch 7: 9140 / 10000
Epoch 8: 9161 / 10000
Epoch 9: 9154 / 10000
Epoch 10: 9165 / 10000
Epoch 11: 9145 / 10000
Epoch 12: 9162 / 10000
Epoch 13: 9140 / 10000
Epoch 14: 9145 / 10000
Epoch 15: 9149 / 10000
Epoch 16: 9150 / 10000
Epoch 17: 9164 / 10000
Epoch 18: 9161 / 10000
Epoch 19: 9163 / 10000
Epoch 20: 9130 / 10000
Epoch 21: 9141 / 10000
Epoch 22: 9175 / 10000
Epoch 23: 9146 / 10000
Epoch 24: 9168 / 10000
Epoch 25: 9120 / 10000
Epoch 26: 9183 / 10000
Epoch 27: 9174 / 10000
Epoch 28: 9151 / 10000
Epoch 29: 9181 / 10000


In [55]:
training_data , validation_data , test_data = load_data_wrapper()
net = Network([784,30, 10])
net.SGD(training_data, 30, 20, 3.0, test_data=test_data)   ####mini batche size 20

Epoch 0: 8851 / 10000
Epoch 1: 9089 / 10000
Epoch 2: 9225 / 10000
Epoch 3: 9264 / 10000
Epoch 4: 9304 / 10000
Epoch 5: 9340 / 10000
Epoch 6: 9387 / 10000
Epoch 7: 9366 / 10000
Epoch 8: 9394 / 10000
Epoch 9: 9418 / 10000
Epoch 10: 9418 / 10000
Epoch 11: 9421 / 10000
Epoch 12: 9435 / 10000
Epoch 13: 9441 / 10000
Epoch 14: 9445 / 10000
Epoch 15: 9451 / 10000
Epoch 16: 9451 / 10000
Epoch 17: 9463 / 10000
Epoch 18: 9456 / 10000
Epoch 19: 9484 / 10000
Epoch 20: 9452 / 10000
Epoch 21: 9467 / 10000
Epoch 22: 9462 / 10000
Epoch 23: 9493 / 10000
Epoch 24: 9455 / 10000
Epoch 25: 9484 / 10000
Epoch 26: 9483 / 10000
Epoch 27: 9489 / 10000
Epoch 28: 9474 / 10000
Epoch 29: 9485 / 10000


In [57]:
training_data , validation_data , test_data = load_data_wrapper()
net = Network([784,30, 10])
net.SGD(training_data, 30, 1, 3.0, test_data=test_data)    #####mini batch size = 1

Epoch 0: 8210 / 10000
Epoch 1: 8040 / 10000
Epoch 2: 8403 / 10000


c:\users\91958\envs\pytorch\lib\site-packages\ipykernel_launcher.py:90: RuntimeWarning: overflow encountered in exp


Epoch 3: 8035 / 10000
Epoch 4: 8371 / 10000
Epoch 5: 8271 / 10000
Epoch 6: 8448 / 10000
Epoch 7: 8415 / 10000
Epoch 8: 8659 / 10000
Epoch 9: 8690 / 10000
Epoch 10: 8596 / 10000
Epoch 11: 8629 / 10000
Epoch 12: 8630 / 10000
Epoch 13: 8563 / 10000
Epoch 14: 8439 / 10000
Epoch 15: 8522 / 10000
Epoch 16: 8712 / 10000
Epoch 17: 8647 / 10000
Epoch 18: 8421 / 10000
Epoch 19: 8643 / 10000
Epoch 20: 7926 / 10000
Epoch 21: 8647 / 10000
Epoch 22: 8658 / 10000
Epoch 23: 8466 / 10000
Epoch 24: 8643 / 10000
Epoch 25: 8595 / 10000
Epoch 26: 8836 / 10000
Epoch 27: 8735 / 10000
Epoch 28: 8629 / 10000
Epoch 29: 8773 / 10000
